# **In this notebook, I will build a baseline model for predicting the aggregated demand from T+1 to T+5**

In [1]:
import pandas as pd
import numpy as np
import geohash2
import os
import tensorflow as tf

import keras 
from keras import backend as K
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
tf.enable_eager_execution()

Using TensorFlow backend.


In [2]:
data_dir = '/home/angps/Documents/GrabChallenge/Traffic Management/Data'

In [3]:
df = pd.read_csv(os.path.join(data_dir,'cleaned_training.csv'))

In [4]:
df.head()

,geohash6,day,demand,latitude,longitude,latitude_error,longitude_error,Hour,Minute,Period
0,qp03wc,18,0.020072,-5.353088,90.653687,0.002747,0.005493,20,0,1712
1,qp03pn,10,0.024721,-5.413513,90.664673,0.002747,0.005493,14,30,922
2,qp09sw,9,0.102821,-5.325623,90.906372,0.002747,0.005493,6,15,793
3,qp0991,32,0.088755,-5.353088,90.752563,0.002747,0.005493,5,0,2996
4,qp090q,15,0.074468,-5.413513,90.719604,0.002747,0.005493,4,0,1360


## **Naive Forecast**

Let's see the baseline T+1 forecast RMSE if we use naive forecast (where T+1 is predicted to be the same as T)

In [5]:
df = df.sort_values(['geohash6','Period'])
df.head()

,geohash6,day,demand,latitude,longitude,latitude_error,longitude_error,Hour,Minute,Period
2873712,qp02yc,1,0.020592,-5.484924,90.653687,0.002747,0.005493,2,45,11
2643238,qp02yc,1,0.010292,-5.484924,90.653687,0.002747,0.005493,3,0,12
1791986,qp02yc,1,0.006676,-5.484924,90.653687,0.002747,0.005493,4,0,16
4022333,qp02yc,1,0.003822,-5.484924,90.653687,0.002747,0.005493,4,30,18
3848343,qp02yc,1,0.011131,-5.484924,90.653687,0.002747,0.005493,6,45,27


In [6]:
demand = df['demand'].values
pred = demand[:-1]
actual = demand[1:]

In [7]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

x = rmse(actual, pred)
print('RMSE for naive forecast for T+1: ') 
print(x)

RMSE for naive forecast for T+1: 
tf.Tensor(0.03464925396052445, shape=(), dtype=float64)


**The RMSE for T+1 using naive forecast is about 3.5%, which is pretty good.**

## **Baseline Model**

**We will only use data from day 15 onwards so every data point has a previous full 14 day demand data. We would also split the dataset 80-20 for training and test set. A simple LSTM will be used.**

In [8]:
def dayhourmin_to_period(day, hour, minute):
    return ((day-1) * 24 * 4) + (hour * 4) + minute//15

def period_to_dayhourmin(period):
    day = period//96 + 1
    hour = (period - (day-1) * 96)//4
    minute = (period - ((day-1) * 96) - (hour*4)) * 15
    return (day, hour, minute)

def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

def onehot(day):
    day = (day-1) % 7
    ls = [0] * 7
    ls[day] = 1
    return np.array(ls) 

df = encode(df, 'Hour', 23)
df = encode(df, 'Minute', 45)


def get_feats(df, geohash, period, num_past_per):
    temp_df = df[(df['geohash6']==geohash) & (df['Period']<=period)].sort_values('Period', ascending=False).reset_index(drop=True, inplace=False)
    past_demands = temp_df.demand.values[1:num_past_per+1].astype(float)
    past_periods = temp_df.Period.values[1:num_past_per+1].astype(int)
    feats = np.array([])
    for i in range(num_past_per):
        day,hour,minute = period_to_dayhourmin(past_periods[i])
        days_feat = onehot(day)
        time_feat = temp_df.loc[i,['Hour_sin', 'Hour_cos', 'Minute_sin', 'Minute_cos']].values.astype(float)
        feat = np.concatenate((np.array([past_demands[i]]), days_feat, time_feat),axis=None)
        feats = np.vstack((feats,feat)) if feats.size else feat
    return feats

def get_demand_from_period(df, geohash, period):
    demand_queried = df[(df['geohash6'] == geohash) & (df['Period']==period)]['demand'].values
    if len(demand_queried) > 0:
        return demand_queried[0]
    else:
        return 0

In [10]:

def data_gen(typ, batch_size, num_past_per=24):
    import time
    if typ == 'train':
        df = train_df
    elif typ == 'val':
        df = val_df
    df_1 = df[(df['day'] > 14) & (df['day'] < 60)].reset_index(drop=True, inplace=False)
    max_rows = len(df_1)
    while True:
        feats = np.zeros((batch_size, num_past_per, 12))
        future_demands = np.zeros((batch_size, 1))
        rows = np.random.choice(a=max_rows, size=batch_size, replace=False)
        for i in range(len(rows)):
            row = rows[i]
            geohash = df_1.loc[row, 'geohash6']
            period = df_1.loc[row, 'Period']
            feats[i] = get_feats(df, geohash, period, num_past_per)
            future_demands[i] = get_demand_from_period(df, geohash, period)
        yield feats, future_demands


In [11]:
def fc_layer(inputs, output_units, batch_norm=True):
    net = tf.keras.layers.Dense(output_units)(inputs)
    if batch_norm:
        net = tf.keras.layers.BatchNormalization()(net)
    net = tf.keras.layers.Activation('relu')(net)
    return net

def baseline_model():
    _input = tf.keras.layers.Input(shape=(24,12))
    net = tf.keras.layers.CuDNNLSTM(units=32)(_input)
    #net = fc_layer(net, 16)
    net = tf.keras.layers.BatchNormalization()(net)
    net = tf.keras.layers.Activation('relu')(net)
    net = tf.keras.layers.Dense(1, kernel_initializer='normal', activation='relu')(net)
    model = tf.keras.models.Model(inputs=_input, outputs=net)
    return model

model = baseline_model()

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

model.compile(optimizer = "rmsprop", loss = root_mean_squared_error, 
              metrics =["mean_squared_error"])

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 24, 12)            0         
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       (None, 32)                5888      
_________________________________________________________________
batch_normalization_v1 (Batc (None, 32)                128       
_________________________________________________________________
activation (Activation)      (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 6,049
Trainable params: 5,985
Non-trainable params: 64
_________________________________________________________________


In [13]:
#df = df[(df['day'] > 20) & (df['day'] < 60)]
df = df.groupby('geohash6').filter(lambda x: len(x)>600)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=0, stratify=df[['geohash6']])
train_df = train_df.reset_index(drop=True, inplace=False)
val_df = val_df.reset_index(drop=True, inplace=False)

In [14]:
train_df.shape, val_df.shape

((3327996, 14), (832000, 14))

In [15]:
train_gen = data_gen('train', batch_size=32)
val_gen = data_gen('val', batch_size=32)
num_epochs = 5
base_lr = 0.001


def lr_linear_decay(epoch):
    return (base_lr * (1 - (epoch/num_epochs)))

history = model.fit_generator(generator=train_gen,
                              validation_data=val_gen,
                              steps_per_epoch=20,
                              validation_steps=20,
                              max_queue_size=10,
                              epochs=num_epochs,
                              verbose=1)

Epoch 1/5
Instructions for updating:
Use tf.cast instead.
20/20 [==============================] - 740s 37s/step - loss: 0.1025 - mean_squared_error: 0.0363 - val_loss: 0.0946 - val_mean_squared_error: 0.0315
Epoch 2/5
20/20 [==============================] - 519s 26s/step - loss: 0.0840 - mean_squared_error: 0.0259 - val_loss: 0.0973 - val_mean_squared_error: 0.0346
Epoch 3/5
20/20 [==============================] - 517s 26s/step - loss: 0.0754 - mean_squared_error: 0.0181 - val_loss: 0.1029 - val_mean_squared_error: 0.0372
Epoch 4/5
 8/20 [===========>..................] - ETA: 3:48 - loss: 0.0798 - mean_squared_error: 0.0237

KeyboardInterrupt: 

We can see that the simple LSTM model doesn't work well